In [1]:
from langchain import OpenAI, VectorDBQA, LLMChain
from langchain.prompts import PromptTemplate
import os
from dotenv import load_dotenv
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader

In [2]:
import sys
sys.path.append(r"C:\Users\ELAFACRB1\Codice\GitHub\media-chat-service\src\embedding")
from utils import ChromaDBManager, LangChainAI, QDrantDBManager, EmbeddingFunction

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

In [4]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
embedding = EmbeddingFunction('openAI').embedder
vectore_store=qdrantClient = QDrantDBManager(
    url="http://ec2-18-209-145-26.compute-1.amazonaws.com:6333/dashboard",
    port=6333,
    collection_name="rio-rag-platform",
    vector_size=1536,
    embedding=embedding,
    record_manager_url=r"sqlite:///C:\Users\ELAFACRB1\Codice\GitHub\media-chat-service\src\embedding\record_manager_cache.sql"
)
vectore_store_client=vectore_store.vector_store

c:\Users\ELAFACRB1\venvs\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
c:\Users\ELAFACRB1\venvs\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


Collection rio-rag-platform already exists!


In [5]:
COLLECTION_NAME="rio-rag-platform"

In [6]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectore_store_client.as_retriever()
)

In [7]:
query='valore per i clienti'

In [8]:
found_docs = vectore_store_client.similarity_search(query)

AuthenticationError: Incorrect API key provided: sk-dH3K6***************************************jbQ3. You can find your API key at https://platform.openai.com/account/api-keys.

In [ ]:
found_docs

[Document(page_content='una\tsettimana\tdi\tconsolidamento\tin\toperativo\tdei\tcomportamenti attraverso\tmomenti\tdi\tcoaching,\tstrumenti\tdi\tauto\ted eterovalutazione,\tinterviste\tai\tclienti\timmediatamente\tdopo\tla\tloro chiamata\tal\tservizio\tclienti; team\tmeeting\tfra\til\tteam\tleader\t(capo)\te\trep\t(collaboratori)\tper\tfocus sui\trisultati\tpost\tformazione\te\triflessioni\tsulle\tstrategie\tdel\tteam,\tal fine\tdi\tgarantire\tla\tcontinuità\tdell’applicazione\tdei\tcomportamenti sperimentati. 16.5 \t\tIL\tCUSTOMER\tSTORYTELLING\tCOME\tCARATTERISTICA\tVINCENTE\tDI\tUN\tQUALITY LAB In\ttutto\til\tpercorso\tlaboratoriale\tviene\tanalizzato\te\tapplicato\tun\t“insieme”\tdi comportamenti\tattesi\ta\tcomplessità\tcrescente,\tche\tpermettono\tuna\tri- focalizzazione\tsul\tcliente\te\tsulla\tgestione\tdella\trelazione\tattraverso\tun\tconfronto continuo\te\tuna\tcostante\t rilettura\tnarrativa \tdei\tcomportamenti\tdi\tciascun operatore,\tfornendo\tmotivazioni\te\tazioni\tcon

### output parser

In [13]:
from typing import List

from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

In [14]:
# Define your desired data structure.
class Tool(BaseModel):
    name: str = Field(description="name of the tool used for the specified use case")
    description: str = Field(description="description of the tool used for the specified use case")

In [18]:
# And a query intented to prompt a language model to populate the data structure.
query = "Dimmi uno strumento per creare uno storytelling"

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Tool)

In [19]:
prompt = PromptTemplate(
    template="Rispondi alla query dell'utente.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
prompt

PromptTemplate(input_variables=['query'], partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"title": "Name", "description": "name of the tool used for the specified use case", "type": "string"}, "description": {"title": "Description", "description": "description of the tool used for the specified use case", "type": "string"}}, "required": ["name", "description"]}\n```'}, template="Rispondi alla query dell'utente.\n{format_instructions}\n{query}\n")

In [20]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectore_store_client.as_retriever(
        chain_type_kwargs={"prompt": prompt}
    )
)

In [21]:
qa_chain({"query": query})

c:\Users\ELAFACRB1\venvs\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


AuthenticationError: Incorrect API key provided: sk-dH3K6***************************************jbQ3. You can find your API key at https://platform.openai.com/account/api-keys.